In [1]:
!pip uninstall peft -y
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu120
!pip install datasets transformers torch numpy pandas peft
#!pip install --upgrade --force-reinstall peft

Found existing installation: peft 0.8.2
Uninstalling peft-0.8.2:
  Successfully uninstalled peft-0.8.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu120
  Using cached peft-0.8.2-py3-none-any.whl (183 kB)


# **Loading and Evaluating a Foundation Model**

In [2]:
 !pip install -q "datasets==2.15.0"

In [3]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Initialize tokenizer for GPT-2 model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Set the pad token to be the same as the eos (end of sequence) token
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Load a fraction of the IMDb dataset and then split into training and testing subsets
imdb_dataset = load_dataset('imdb', split='train[:10%]').train_test_split(test_size=0.3, shuffle=True, seed=42)

# Prepare for tokenization
processed_data = {}

# Function to tokenize text
def tokenize_text(examples):
    return gpt2_tokenizer(examples['text'], truncation=True, max_length=512)

# Process each split in the dataset
for data_split in imdb_dataset.keys():
    try:
        # Apply tokenization and store in processed_data
        processed_data[data_split] = imdb_dataset[data_split].map(tokenize_text, batched=True)
    except Exception as e:
        # Handle potential exceptions during tokenization
        print(f"Could not tokenize {data_split}: {str(e)}")

# Update tokenizer with additional special tokens, if necessary
additional_tokens = gpt2_tokenizer.add_special_tokens({'cls_token': '[CLS]'})

# Displaying a snippet of the tokenized dataset to verify
print("Sample Tokenized Output:", processed_data['train'][0]['input_ids'])

# Detailed view of the first processed training example
print("First Training Example Tokenized Input IDs:", processed_data['train'][0]['input_ids'])


/home/saifm/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Map:   0%|          | 0/750 [00:00<?, ? examples/s]

Sample Tokenized Output: [818, 616, 19695, 4459, 11, 428, 2196, 286, 262, 1049, 28023, 54, 56, 10530, 468, 691, 734, 1243, 1016, 329, 340, 532, 7039, 710, 43692, 290, 262, 1109, 326, 612, 318, 783, 257, 18976, 2196, 351, 262, 2656, 4226, 13, 357, 11380, 42100, 6484, 318, 922, 284, 2342, 2014, 1537, 284, 502, 326, 338, 477, 612, 318, 13, 4042, 286, 262, 3350, 1283, 284, 307, 6155, 832, 262, 905, 532, 609, 2047, 2616, 17630, 468, 645, 2126, 508, 6502, 1107, 318, 290, 645, 4240, 661, 2513, 625, 5850, 1982, 44314, 618, 340, 338, 2826, 416, 4502, 21042, 83, 508, 3073, 588, 339, 1549, 2138, 307, 736, 319, 257, 2318, 38753, 287, 6182, 13, 8982, 10009, 318, 1208, 540, 11, 475, 326, 45678, 468, 284, 467, 290, 314, 2497, 1365, 15360, 287, 15217, 1837, 40735, 13, 1081, 314, 4750, 11, 340, 338, 922, 284, 423, 257, 2196, 286, 262, 3800, 4226, 783, 11, 475, 314, 2911, 262, 1862, 503, 612, 11, 508, 423, 1239, 1775, 257, 10530, 11, 23917, 6, 51, 5052, 606, 477, 416, 428, 13]
First Training Example Tok

In [4]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
import numpy as np
import pandas as pd

# Load and prepare datasets
imdb_subset = load_dataset("imdb", split="train[:50%]").train_test_split(test_size=0.3, shuffle=True, seed=23)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Tokenize the dataset
def tokenize_batch(examples):
    return gpt2_tokenizer(examples['text'], truncation=True, padding=True)

tokenized_datasets = {split: imdb_subset[split].map(tokenize_batch, batched=True) for split in imdb_subset}

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1})
model.config.pad_token_id = model.config.eos_token_id

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

# Adjust the token embeddings size
model.resize_token_embeddings(len(gpt2_tokenizer))

# Setup training
training_args = TrainingArguments(
    output_dir="./model_outputs",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=gpt2_tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=gpt2_tokenizer),
    compute_metrics=compute_accuracy,
)

# Start training and evaluation
trainer.train()
evaluation_results = trainer.evaluate()

# Generate predictions for analysis
test_dataset = tokenized_datasets["test"]
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Prepare and display a sample dataframe for inspection
df_samples = pd.DataFrame({'text': test_dataset['text'], 'actual_label': test_dataset['label'], 'predicted_label': predicted_labels})
df_samples['text'] = df_samples['text'].apply(lambda x: x.replace("<br />", " "))
df_samples.head()


Map:   0%|          | 0/8750 [00:00<?, ? examples/s]

Map:   0%|          | 0/3750 [00:00<?, ? examples/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/home/saifm/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/home/saifm/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


text  actual_label  \
0  The most embarrassing moment in this film is w...             0   
1  There is a lot of obvious hype associated with...             0   
2  30 seconds into the opening credits, I had thi...             0   
3  Well then, what is it?! I found Nicholson's ch...             0   
4  Johnny and Jeremy are vampires of sorts. Minus...             0   

   predicted_label  
0                1  
1                1  
2                1  
3                1  
4                1

## **Performing Parameter-Efficient Fine Tuning**

In [7]:
from transformers import AutoModelForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, pipeline
from peft import PeftModelForSequenceClassification, LoraConfig, TaskType
from datasets import load_dataset
import numpy as np
import pandas as pd

# Step 1: Initialize the tokenizer for GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Step 2: Load and prepare the IMDb dataset, then tokenize
imdb_dataset = load_dataset('imdb', split='train[:10%]').train_test_split(test_size=0.3, shuffle=True, seed=42)
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=512)
tokenized_datasets = {split: imdb_dataset[split].map(tokenize_function, batched=True) for split in imdb_dataset}

# Step 3: Initialize the GPT-2 model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1})
model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(tokenizer))

# Step 4: Configure PEFT and enhance the base model
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
peft_model = PeftModelForSequenceClassification(model, peft_config)

# Ensure all base model parameters are trainable
for param in model.base_model.parameters():
    param.requires_grad = True

# Step 5: Training setup and execution
def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir="./model_outputs/peft_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_accuracy,
)

trainer.train()
trainer.evaluate()




Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/saifm/anaconda3/lib/python3.8/site-packages/peft/tuners/lora/layer.py:711: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Checkpoint destination directory ./model_outputs/peft_model/checkpoint-37 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/home/saifm/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.10893378406763077,
 'eval_accuracy': 0.9893333333333333,
 'eval_runtime': 28.0558,
 'eval_samples_per_second': 26.732,
 'eval_steps_per_second': 0.57,
 'epoch': 1.0}

# Performing Inference with a PEFT Model

In [8]:
# Step 6: Use the fine-tuned model for sentiment analysis
classifier = pipeline("sentiment-analysis", model="./model_outputs/peft_model", tokenizer=tokenizer)

texts = [
    "The storytelling was masterful, keeping me engaged from start to finish.",
    "Absolutely breathtaking visuals, paired with a story that tugs at your heartstrings.",
    "A hilarious adventure that had me laughing out loud multiple times.",
    "The performances were stellar, breathing life into every character.",
    "An underrated gem that delivers on all fronts, exceeding expectations.",
    "The soundtrack complements the movie perfectly, enhancing every scene.",
    "A dull plot that meanders aimlessly, wasting its potential.",
    "Filled with cliches, the movie fails to bring anything new to the table.",
    "Disjointed editing and pacing issues make it a chore to sit through.",
    "The dialogue felt unnatural and forced, detracting from the immersion.",
    "An uninspired sequel that fails to capture the magic of its predecessor.",
    "Lackluster performances make it difficult to care about the characters."
]


results = classifier(texts)
for i, text in enumerate(texts):
    print(f"Text {i+1}: {text}\nClassification: {results[i]}\n")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Text 1: The storytelling was masterful, keeping me engaged from start to finish.
Classification: {'label': 'LABEL_0', 'score': 0.5158314108848572}

Text 2: Absolutely breathtaking visuals, paired with a story that tugs at your heartstrings.
Classification: {'label': 'LABEL_1', 'score': 0.6025356650352478}

Text 3: A hilarious adventure that had me laughing out loud multiple times.
Classification: {'label': 'LABEL_1', 'score': 0.723960816860199}

Text 4: The performances were stellar, breathing life into every character.
Classification: {'label': 'LABEL_0', 'score': 0.6323111653327942}

Text 5: An underrated gem that delivers on all fronts, exceeding expectations.
Classification: {'label': 'LABEL_1', 'score': 0.7032448053359985}

Text 6: The soundtrack complements the movie perfectly, enhancing every scene.
Classification: {'label': 'LABEL_1', 'score': 0.5825276970863342}

Text 7: A dull plot that meanders aimlessly, wasting its potential.
Classification: {'label': 'LABEL_0', 'score': 0